In [1]:
#Load Libraries

import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

C:\Users\sonne\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Loading the dataset

house = pd.read_csv("E:/datasets/kc_house_data.csv")

In [3]:
# Missing value Analysis
print("count of missing values")
missing_val = pd.DataFrame(house.isnull().sum())
print(missing_val)

,0
id,0
date,0
price,0
bedrooms,0
bathrooms,0
sqft_living,0
sqft_lot,0
floors,0
waterfront,0
view,0


In [ ]:
#Columns of the dataset

In [4]:
print('Columns of the dataset')
print(house.columns)

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [ ]:
#Converting into proper datatype

In [15]:
house['id']=house['id'].astype(object)
house['waterfront']=house['waterfront'].astype(object)
house['view']=house['view'].astype(object)
house['condition']=house['condition'].astype(object)
house['grade']=house['grade'].astype(object)
house['zipcode']=house['zipcode'].astype(object)
house['yr_renovated']=house['yr_renovated'].astype(object)
house['yr_built']=house['yr_built'].astype(object)

In [16]:
#Assigning levels to the categories
lis = []
for i in range(0, house.shape[1]):
    if(house.iloc[:,i].dtypes == 'object'):
        house.iloc[:,i] = pd.Categorical(house.iloc[:,i])
        house.iloc[:,i] = house.iloc[:,i].cat.codes 
        house.iloc[:,i] = house.iloc[:,i].astype('object')
        lis.append(house.columns[i])
        

In [17]:
# Extracting numerical variables

In [18]:
lis1=[]
for i in range(0, house.shape[1]):
    if(house.iloc[:,i].dtypes != 'object'):
        lis1.append(house.columns[i])

In [19]:
print('Extracting numerical variables')
print(lis1)

['price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'sqft_above',
 'sqft_basement',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [ ]:
# Box plots of all numerical variables

In [20]:
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange, uniform
import plotly

In [21]:
tr=[]
for i in lis1:
    tr.append(go.Box(y=house[i]))

plotly.offline.plot(tr,filename='box_plotly.html')

'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\box_plotly.html'

In [22]:
#Detect and remove outliers
for i in lis1:
    
    # Extract quartiles
    q75, q25 = np.percentile(house.loc[:,i], [75,25])

    # Calculate IQR
    iqr = q75 - q25

    # Calculate inner and outer fence
    minimum = q25 - (iqr*1.5)
    maximum = q75 + (iqr*1.5)

    # #Replace with NA
    house.loc[house.loc[:,i] < minimum,i] = np.nan
    house.loc[house.loc[:,i] > maximum,i] = np.nan


In [ ]:
# removing the rows which have null values

In [23]:
house=house.dropna()

In [27]:
#Correlation plot

house_corr = house.loc[:,lis1]
dat=house_corr.corr().values
cc=[go.Heatmap(z=dat,colorscale="Viridis",x=house_corr.columns,y=house_corr.columns)]
plotly.offline.plot(cc,filename='correlation_plotly.html')

'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\correlation_plotly.html'

In [28]:
#Chisquare test of independence

#Extract categorical variables
print("Categorical variables")
cat_names = lis
print(cat_names)

['id',
 'date',
 'waterfront',
 'view',
 'condition',
 'grade',
 'yr_built',
 'yr_renovated',
 'zipcode']

In [29]:
#Printing chi square values
print("Results of chi-square test")
for i in cat_names:
    print(i)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(house['price'], house[i]))
    print(p)

id
0.9999980648526372
date
4.968185512829421e-16
waterfront
3.3617488288706776e-37
view
1.4383342423329998e-16
condition
4.353581408408281e-83
grade
0.0
yr_built
1.0
yr_renovated
1.1600724930992988e-101
zipcode
2.196968315959122e-106


In [30]:
# Dimension reduction

house = house.drop(['yr_built', 'grade', 'id', 'sqft_living15', 'sqft_lot15',
              'sqft_above'], axis=1)

In [31]:
# Extracting numerical variables

lis3=[]
for i in range(0, house.shape[1]):
    if(house.iloc[:,i].dtypes != 'object'):
        lis3.append(house.columns[i])
lis3

['price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'sqft_basement',
 'lat',
 'long']

In [ ]:
# Plotting histograms to show the distribution of data

In [32]:
from plotly import tools
tr=[]
for i in lis3:
    tr.append(go.Histogram(x=house[i]))
fig = tools.make_subplots(rows=3, cols=3)
fig.append_trace(tr[0], 1, 1)
fig.append_trace(tr[1], 1, 2)
fig.append_trace(tr[2], 1, 3)
fig.append_trace(tr[3], 2, 1)
fig.append_trace(tr[4], 2, 2)
fig.append_trace(tr[5], 2, 3)
fig.append_trace(tr[6], 3, 1)
fig.append_trace(tr[7], 3, 2)
fig.append_trace(tr[8], 3, 3)
plotly.offline.plot(fig,filename='histogram_plotly.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\histogram_plotly.html'

In [33]:
#Divide data into train and test

X = house[['date','bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition',
           'sqft_basement','yr_renovated', 'zipcode', 'lat', 'long']]
Y = house[['price']]

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2)

In [34]:
# Feature Scaling

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
y_train = min_max_scaler.fit_transform(y_train)
X_test = min_max_scaler.fit_transform(X_test)
y_test = min_max_scaler.fit_transform(y_test)

In [35]:
# Converting into tensor using torch

X=torch.tensor(X_train, dtype=torch.float)
y=torch.tensor(y_train, dtype=torch.float)
X1=torch.tensor(X_test, dtype=torch.float)
y1=torch.tensor(y_test, dtype=torch.float)

In [36]:
# Defining input size, hidden layers size, output size and batch size respectively

n_in, n_h1, n_h2, n_h3, n_h4, n_out, batch_size = 14, 20, 40, 50, 30, 1, 10

In [37]:
# Create a model

model = nn.Sequential(nn.Linear(n_in, n_h3),
                     nn.ReLU(),
                     #nn.Linear(n_h1, n_h2),
                     #nn.ReLU(),
                     #nn.Linear(n_h2, n_h3),
                     #nn.ReLU(),
                     #nn.Linear(n_h1, n_h4),
                     #nn.ReLU(),
                     nn.Linear(n_h3,n_out),
                     nn.Sigmoid())

In [38]:
# Construct the loss function

criterion = torch.nn.MSELoss()

# Construct the optimizer (Stochastic Gradient Descent in this case)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Gradient Descent
epoch_list=[]
loss_list=[]
for epoch in range(5000):
    epoch_list.append(epoch)
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(X)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print('epoch: ', epoch,' loss: ', loss.item())
    loss_list.append(loss)
    
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

epoch:  0  loss:  0.05998914688825607
epoch:  1  loss:  0.05874641612172127
epoch:  2  loss:  0.057549092918634415
epoch:  3  loss:  0.056397367268800735
epoch:  4  loss:  0.05529087409377098
epoch:  5  loss:  0.05422959104180336
epoch:  6  loss:  0.05321300029754639
epoch:  7  loss:  0.05224083736538887
epoch:  8  loss:  0.05131232738494873
epoch:  9  loss:  0.05042634904384613
epoch:  10  loss:  0.049582041800022125
epoch:  11  loss:  0.0487784743309021
epoch:  12  loss:  0.04801470413804054
epoch:  13  loss:  0.04728969559073448
epoch:  14  loss:  0.04660274088382721
epoch:  15  loss:  0.04595276340842247
epoch:  16  loss:  0.045339010655879974
epoch:  17  loss:  0.04476037621498108
epoch:  18  loss:  0.044215813279151917
epoch:  19  loss:  0.04370427131652832
epoch:  20  loss:  0.04322493076324463
epoch:  21  loss:  0.04277678206562996
epoch:  22  loss:  0.042359013110399246
epoch:  23  loss:  0.0419703908264637
epoch:  24  loss:  0.04161008074879646
epoch:  25  loss:  0.0412768870

epoch:  210  loss:  0.01920262910425663
epoch:  211  loss:  0.019136887043714523
epoch:  212  loss:  0.019071558490395546
epoch:  213  loss:  0.019006652757525444
epoch:  214  loss:  0.01894214190542698
epoch:  215  loss:  0.01887805387377739
epoch:  216  loss:  0.01881440170109272
epoch:  217  loss:  0.01875118352472782
epoch:  218  loss:  0.018688388168811798
epoch:  219  loss:  0.01862601935863495
epoch:  220  loss:  0.018564103171229362
epoch:  221  loss:  0.018502628430724144
epoch:  222  loss:  0.018441585823893547
epoch:  223  loss:  0.01838100142776966
epoch:  224  loss:  0.018320869654417038
epoch:  225  loss:  0.018261168152093887
epoch:  226  loss:  0.018201889470219612
epoch:  227  loss:  0.0181430634111166
epoch:  228  loss:  0.018084678798913956
epoch:  229  loss:  0.018026718869805336
epoch:  230  loss:  0.017969176173210144
epoch:  231  loss:  0.01791204698383808
epoch:  232  loss:  0.017855364829301834
epoch:  233  loss:  0.017799152061343193
epoch:  234  loss:  0.0177

epoch:  417  loss:  0.013151375576853752
epoch:  418  loss:  0.013141069561243057
epoch:  419  loss:  0.01313083991408348
epoch:  420  loss:  0.013120672665536404
epoch:  421  loss:  0.013110524043440819
epoch:  422  loss:  0.01310040894895792
epoch:  423  loss:  0.013090300373733044
epoch:  424  loss:  0.013080254197120667
epoch:  425  loss:  0.013070296496152878
epoch:  426  loss:  0.013060429133474827
epoch:  427  loss:  0.013050624169409275
epoch:  428  loss:  0.013040832243859768
epoch:  429  loss:  0.013031050562858582
epoch:  430  loss:  0.013021353632211685
epoch:  431  loss:  0.013011698611080647
epoch:  432  loss:  0.013002024032175541
epoch:  433  loss:  0.012992334552109241
epoch:  434  loss:  0.012982640415430069
epoch:  435  loss:  0.012972935102880001
epoch:  436  loss:  0.012963217683136463
epoch:  437  loss:  0.012953539378941059
epoch:  438  loss:  0.012943868525326252
epoch:  439  loss:  0.012934247963130474
epoch:  440  loss:  0.012924659997224808
epoch:  441  loss:

epoch:  621  loss:  0.01138791162520647
epoch:  622  loss:  0.011378033086657524
epoch:  623  loss:  0.011368170380592346
epoch:  624  loss:  0.011358354240655899
epoch:  625  loss:  0.01134858001023531
epoch:  626  loss:  0.011338743381202221
epoch:  627  loss:  0.011328919790685177
epoch:  628  loss:  0.011319112963974476
epoch:  629  loss:  0.011309360153973103
epoch:  630  loss:  0.011299638077616692
epoch:  631  loss:  0.01128996629267931
epoch:  632  loss:  0.01128037553280592
epoch:  633  loss:  0.011270829476416111
epoch:  634  loss:  0.01126131508499384
epoch:  635  loss:  0.011251741088926792
epoch:  636  loss:  0.011242217384278774
epoch:  637  loss:  0.011232768185436726
epoch:  638  loss:  0.01122333575040102
epoch:  639  loss:  0.011213897727429867
epoch:  640  loss:  0.011204446665942669
epoch:  641  loss:  0.011195031926035881
epoch:  642  loss:  0.011185646057128906
epoch:  643  loss:  0.011176245287060738
epoch:  644  loss:  0.011166826821863651
epoch:  645  loss:  0.

epoch:  829  loss:  0.00977382156997919
epoch:  830  loss:  0.009767791256308556
epoch:  831  loss:  0.00976177304983139
epoch:  832  loss:  0.009755773469805717
epoch:  833  loss:  0.009749782271683216
epoch:  834  loss:  0.009743795730173588
epoch:  835  loss:  0.009737823158502579
epoch:  836  loss:  0.009731878526508808
epoch:  837  loss:  0.009725932031869888
epoch:  838  loss:  0.009720022790133953
epoch:  839  loss:  0.009714122861623764
epoch:  840  loss:  0.009708221070468426
epoch:  841  loss:  0.009702350944280624
epoch:  842  loss:  0.009696497581899166
epoch:  843  loss:  0.009690661914646626
epoch:  844  loss:  0.009684860706329346
epoch:  845  loss:  0.009679071605205536
epoch:  846  loss:  0.009673300199210644
epoch:  847  loss:  0.00966754462569952
epoch:  848  loss:  0.009661808609962463
epoch:  849  loss:  0.009656081907451153
epoch:  850  loss:  0.009650388732552528
epoch:  851  loss:  0.009644711390137672
epoch:  852  loss:  0.009639040566980839
epoch:  853  loss: 

...

epoch:  4186  loss:  0.005722819361835718
epoch:  4187  loss:  0.005722420755773783
epoch:  4188  loss:  0.005721946246922016
epoch:  4189  loss:  0.005721312016248703
epoch:  4190  loss:  0.005720656830817461
epoch:  4191  loss:  0.005720056127756834
epoch:  4192  loss:  0.005719571840018034
epoch:  4193  loss:  0.005719218868762255
epoch:  4194  loss:  0.0057189809158444405
epoch:  4195  loss:  0.005718844011425972
epoch:  4196  loss:  0.005718806758522987
epoch:  4197  loss:  0.0057188537903130054
epoch:  4198  loss:  0.005718932952731848
epoch:  4199  loss:  0.005719002336263657
epoch:  4200  loss:  0.005718941334635019
epoch:  4201  loss:  0.005718681961297989
epoch:  4202  loss:  0.005718220490962267
epoch:  4203  loss:  0.005717621650546789
epoch:  4204  loss:  0.005716993939131498
epoch:  4205  loss:  0.005716444458812475
epoch:  4206  loss:  0.005716012790799141
epoch:  4207  loss:  0.0057156989350914955
epoch:  4208  loss:  0.005715491250157356
epoch:  4209  loss:  0.00571535

epoch:  4381  loss:  0.005678181070834398
epoch:  4382  loss:  0.005677979905158281
epoch:  4383  loss:  0.005677776876837015
epoch:  4384  loss:  0.005677569191902876
epoch:  4385  loss:  0.0056773643009364605
epoch:  4386  loss:  0.005677174776792526
epoch:  4387  loss:  0.005676985252648592
epoch:  4388  loss:  0.005676804110407829
epoch:  4389  loss:  0.005676628556102514
epoch:  4390  loss:  0.005676460452377796
epoch:  4391  loss:  0.005676300264894962
epoch:  4392  loss:  0.005676162429153919
epoch:  4393  loss:  0.005676054861396551
epoch:  4394  loss:  0.005675953812897205
epoch:  4395  loss:  0.005675880238413811
epoch:  4396  loss:  0.005675808992236853
epoch:  4397  loss:  0.005675712134689093
epoch:  4398  loss:  0.005675577558577061
epoch:  4399  loss:  0.005675402004271746
epoch:  4400  loss:  0.00567521620541811
epoch:  4401  loss:  0.0056749871000647545
epoch:  4402  loss:  0.005674720276147127
epoch:  4403  loss:  0.005674414336681366
epoch:  4404  loss:  0.0056740846

epoch:  4581  loss:  0.005643515381962061
epoch:  4582  loss:  0.005643086973577738
epoch:  4583  loss:  0.005642641801387072
epoch:  4584  loss:  0.005642251577228308
epoch:  4585  loss:  0.0056419516913592815
epoch:  4586  loss:  0.005641738418489695
epoch:  4587  loss:  0.0056416005827486515
epoch:  4588  loss:  0.005641531199216843
epoch:  4589  loss:  0.005641509313136339
epoch:  4590  loss:  0.0056415144354105
epoch:  4591  loss:  0.0056415097787976265
epoch:  4592  loss:  0.005641462281346321
epoch:  4593  loss:  0.0056413705460727215
epoch:  4594  loss:  0.005641227588057518
epoch:  4595  loss:  0.005641021300107241
epoch:  4596  loss:  0.005640780087560415
epoch:  4597  loss:  0.005640529096126556
epoch:  4598  loss:  0.005640259478241205
epoch:  4599  loss:  0.005639966577291489
epoch:  4600  loss:  0.005639670416712761
epoch:  4601  loss:  0.005639392416924238
epoch:  4602  loss:  0.005639147479087114
epoch:  4603  loss:  0.005638951901346445
epoch:  4604  loss:  0.005638793

epoch:  4778  loss:  0.005611542146652937
epoch:  4779  loss:  0.00561137218028307
epoch:  4780  loss:  0.005611238535493612
epoch:  4781  loss:  0.005611138883978128
epoch:  4782  loss:  0.00561105739325285
epoch:  4783  loss:  0.005611013155430555
epoch:  4784  loss:  0.005610997788608074
epoch:  4785  loss:  0.0056109922006726265
epoch:  4786  loss:  0.005611006170511246
epoch:  4787  loss:  0.0056110224686563015
epoch:  4788  loss:  0.005611046217381954
epoch:  4789  loss:  0.005611027590930462
epoch:  4790  loss:  0.005610977299511433
epoch:  4791  loss:  0.005610853433609009
epoch:  4792  loss:  0.00561064388602972
epoch:  4793  loss:  0.005610340274870396
epoch:  4794  loss:  0.005610010586678982
epoch:  4795  loss:  0.005609648302197456
epoch:  4796  loss:  0.0056093111634254456
epoch:  4797  loss:  0.005609035957604647
epoch:  4798  loss:  0.005608823616057634
epoch:  4799  loss:  0.005608673673123121
epoch:  4800  loss:  0.005608584266155958
epoch:  4801  loss:  0.00560855166

epoch:  4973  loss:  0.005584561266005039
epoch:  4974  loss:  0.005584311205893755
epoch:  4975  loss:  0.005584042519330978
epoch:  4976  loss:  0.005583785008639097
epoch:  4977  loss:  0.00558354239910841
epoch:  4978  loss:  0.005583328194916248
epoch:  4979  loss:  0.005583138205111027
epoch:  4980  loss:  0.005582964047789574
epoch:  4981  loss:  0.005582806188613176
epoch:  4982  loss:  0.005582652986049652
epoch:  4983  loss:  0.005582503043115139
epoch:  4984  loss:  0.005582364276051521
epoch:  4985  loss:  0.005582231562584639
epoch:  4986  loss:  0.005582112818956375
epoch:  4987  loss:  0.005582017824053764
epoch:  4988  loss:  0.005581966135650873
epoch:  4989  loss:  0.005581966135650873
epoch:  4990  loss:  0.005582061596214771
epoch:  4991  loss:  0.00558221573010087
epoch:  4992  loss:  0.005582468118518591
epoch:  4993  loss:  0.0055827368050813675
epoch:  4994  loss:  0.005583065561950207
epoch:  4995  loss:  0.00558325694873929
epoch:  4996  loss:  0.0055832457728

In [40]:
# Predict the target variable of test dataset

prediction=model(X1)

In [41]:
# Loss in test dataset

import numpy
loss_test = criterion(prediction, y1)
print('Loss in test dataset')
print(test.detach().numpy().tolist())

0.00627808878198266

In [42]:
pred_arr=prediction.detach().numpy()
y1_arr=y1.detach().numpy()

In [43]:
#predicted output

y_pred_act = min_max_scaler.inverse_transform(pred_arr)

In [44]:
from scipy import stats
# Generate linear fit

slope, intercept, r_value, p_value, std_err = stats.linregress(y1_arr.flatten(),pred_arr.flatten())
line = slope*y1_arr.flatten()+intercept

In [45]:
#plotting actual vs predicted

trace=go.Scatter(
    x = pred_arr.flatten(),
    y = y1_arr.flatten(),
    mode = 'markers'
)
trace1=go.Scatter(
    x = line.flatten(),
    y = y1_arr.flatten(),
    mode = 'lines+markers'
)
dat=[trace,trace1]
plotly.offline.plot(dat,filename='actual_vs_predicted_pytorch.html')

'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\actual_vs_predicted_pytorch.html'

In [96]:
#plotting epoch vs loss

In [46]:
lis4=[]
for i in loss_list:
    x=i.detach().numpy()
    x=x.tolist()
    lis4.append(x)

In [47]:
trace2=go.Scatter(
    x = epoch_list,
    y = lis4,
    mode = 'lines+markers'
)
dat=[trace2]
plotly.offline.plot(dat,filename='epoch_vs_loss_pytorch.html')

'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\epoch_vs_loss_pytorch.html'

In [ ]:
############################################################################################################################

In [ ]:
# With Keras

In [48]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

Using TensorFlow backend.


In [49]:
# create model

model = Sequential()
model.add(Dense(20, input_dim=14, kernel_initializer='normal', activation='relu'))
model.add(Dense(40, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(30, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

In [50]:
# Compile model

model.compile(loss='mean_squared_error', optimizer='adam')

In [51]:
# Fit the model

mod=model.fit(X_train, y_train, epochs=150, batch_size=10).history

Epoch 1/150
13624/13624 [==============================] - 4s 306us/step - loss: 0.0204
Epoch 2/150
13624/13624 [==============================] - 2s 137us/step - loss: 0.0147
Epoch 3/150
13624/13624 [==============================] - 2s 137us/step - loss: 0.0144
Epoch 4/150
13624/13624 [==============================] - 2s 139us/step - loss: 0.0133
Epoch 5/150
13624/13624 [==============================] - 2s 136us/step - loss: 0.0115
Epoch 6/150
13624/13624 [==============================] - 2s 138us/step - loss: 0.0103
Epoch 7/150
13624/13624 [==============================] - 2s 138us/step - loss: 0.0097
Epoch 8/150
13624/13624 [==============================] - 2s 142us/step - loss: 0.0089
Epoch 9/150
13624/13624 [==============================] - 2s 149us/step - loss: 0.0082
Epoch 10/150
13624/13624 [==============================] - 2s 167us/step - loss: 0.0079
Epoch 11/150
13624/13624 [==============================] - 2s 181us/step - loss: 0.0076
Epoch 12/150
13624/13624 [====

13624/13624 [==============================] - 3s 184us/step - loss: 0.0051
Epoch 94/150
13624/13624 [==============================] - 2s 157us/step - loss: 0.0051
Epoch 95/150
13624/13624 [==============================] - 2s 138us/step - loss: 0.0050
Epoch 96/150
13624/13624 [==============================] - 2s 152us/step - loss: 0.0051
Epoch 97/150
13624/13624 [==============================] - 2s 147us/step - loss: 0.0051
Epoch 98/150
13624/13624 [==============================] - 2s 142us/step - loss: 0.0051
Epoch 99/150
13624/13624 [==============================] - 2s 151us/step - loss: 0.0050
Epoch 100/150
13624/13624 [==============================] - 2s 145us/step - loss: 0.0051
Epoch 101/150
13624/13624 [==============================] - 2s 141us/step - loss: 0.0050
Epoch 102/150
13624/13624 [==============================] - 2s 144us/step - loss: 0.0050
Epoch 103/150
13624/13624 [==============================] - 2s 141us/step - loss: 0.0050
Epoch 104/150
13624/13624 [===

In [52]:
# evaluate the model
print('Loss in test dataset')
scores = model.evaluate(X_test, y_test)
print(scores)

Loss in test dataset
3407/3407 [==============================] - 0s 101us/step
0.008922065422321594


In [53]:
#predicting the target variable for the test dataset

pred_kera=model.predict(X_test)

In [ ]:
#plotting actual vs predicted

In [54]:
from scipy import stats
# Generated linear fit

slope, intercept, r_value, p_value, std_err = stats.linregress(y_test.flatten(),pred_kera.flatten())
line = slope*y_test.flatten()+intercept

In [55]:
trace=go.Scatter(
    x = pred_kera.flatten(),
    y = y_test.flatten(),
    mode = 'markers'
)
trace1=go.Scatter(
    x = line.flatten(),
    y = y_test.flatten(),
    mode = 'lines+markers'
)
dat=[trace,trace1]
plotly.offline.plot(dat,filename='actual_vs_predicted_keras.html')

'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\actual_vs_predicted_keras.html'

In [190]:
#plotting epoch vs loss

In [56]:
epoch_kera=list(range(150))

In [57]:
trace2=go.Scatter(
    x = epoch_kera,
    y = mod['loss'],
    mode = 'lines+markers'
)
dat=[trace2]
plotly.offline.plot(dat,filename='epoch_vs_loss_keras.html')

'file://C:\\Users\\sonne\\Desktop\\Python_Pro_2\\epoch_vs_loss_keras.html'